Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

In [1]:
using LinearAlgebra, Statistics

using DataFrames
using BenchmarkTools


In [2]:
using Distributed
nprocs = 2
addprocs(nprocs);

In [3]:
@everywhere using NBInclude
@everywhere @nbinclude("solve_model.ipynb")

#importou a função solve_tree

In [4]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj, γ
game_parameters = [0.81, 4.61, 0.09, 0.99, 0.37];

In [5]:
pids = workers()
aux = pids[nprocs]

3

In [6]:
A2 = @spawnat aux solve_tree(data2, game_parameters)

A1 = solve_tree(data1, game_parameters);

A2 = fetch(A2);

  6.739853 seconds (31.35 M allocations: 707.821 MiB, 4.50% gc time)
        From worker 3:	  8.018762 seconds (31.35 M allocations: 707.802 MiB, 3.62% gc time)
6.726298 seconds (30.02 M allocations: 642.090 MiB, 1.53% gc time)
       From worker 3:	  8.506036 seconds (30.02 M allocations: 642.090 MiB, 1.13% gc time)
10.238506 seconds (30.02 M allocations: 642.090 MiB, 0.99% gc time)
       From worker 3:	 10.008801 seconds (30.02 M allocations: 642.090 MiB, 0.78% gc time)
10.218926 seconds (30.02 M allocations: 642.090 MiB, 0.87% gc time)
       From worker 3:	 10.920334 seconds (30.02 M allocations: 642.090 MiB, 0.85% gc time)
11.004492 seconds (30.02 M allocations: 642.090 MiB, 0.91% gc time)
       From worker 3:	 10.967564 seconds (30.02 M allocations: 642.090 MiB, 0.77% gc time)
10.349675 seconds (30.02 M allocations: 642.090 MiB, 0.82% gc time)
       From worker 3:	 10.756288 seconds (30.02 M allocations: 642.090 MiB, 0.75% gc time)
10.523538 seconds (30.02 M allocations: 642.0

Simulation parameters

In [7]:
#θs0, θj0, μ
simulation_parameters = [90.0, 30.0, 7.02];

### Criando funções para simular o jogo

In [8]:
#cost function
function Ct(t, c0=c0, c1=c1)
    
    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function prob_cd(γ, share)

    #probability of a cram down
    return γ * share
end




function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next, pie_share = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2, 3]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
    
    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [9]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == grid)
        
        return grid

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [10]:
function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




# #test
# Hs0 = 30
# Hj0 = 28


# hst = Hs0
# hjt = Hj0

# #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
# lst = hst
# ljt = hjt




# choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [11]:
function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    
    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj
    
    
    c0 = game_parameters[3];
    c1 = c0/30;

    ρ = game_parameters[1];
    λj = game_parameters[4];
    γ = game_parameters[5];

    
    
#     hst = Hs0
#     hjt = Hj0
    
    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(6);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
        
        #proposal ####
        policy, payoff_prop, payment, lm_next, pie_share = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                if(u < prob_cd(γ, pie_share)) #cram down occurs
                    
                    result[prop_index] = payoff_prop
                    result[respondent_index] = payoff_respondent
                    result[3] = 3.0
                    result[4] = t
                    result[5] = observed_proposals
                    result[6] = one(1.0) #dummy equal to one if cram down occurs
                else
                    
                    t+=1

                    hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
                    
                end

            end

        elseif(policy==2.0)
            
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
            
            

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [12]:
#unit testing
simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

6-element Array{Float64,1}:
 0.07153978167813808
 0.34234193610562325
 3.0
 2.0
 2.0
 1.0

In [13]:
#unit testing 
simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])

6-element Array{Float64,1}:
 0.25504758863282295
 0.6577158216394153
 3.0
 5.0
 5.0
 0.0

In [14]:
function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    
    
    
    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster
    
    
    
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 8)

    #últimas entradas de Results são os valores das dívidas
    
    
    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:6] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:6] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end
            
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [15]:
#unit testing
S = 40
N = 66
Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

40×66×8 Array{Float64,3}:
[:, :, 1] =
 0.0711771  0.0662232  0.070177   0.0708478  …  0.259203  0.267835  0.265642
 0.0681899  0.0704187  0.0684875  0.070177      0.259179  0.259482  0.261437
 0.0657124  0.0707231  0.0652442  0.0688534     0.265642  0.26524   0.271675
 0.0695851  0.0702482  0.0662232  0.069302      1.10515   0.258601  0.271958
 0.0703597  0.0707621  0.0662232  0.0726047     0.259179  0.259905  0.265976
 0.0665886  0.0727059  0.0658321  0.0728116  …  0.271504  0.259179  0.265096
 0.0638179  0.0698598  0.0682316  0.0675047     0.259401  0.259191  0.253056
 0.0674745  0.0705247  0.0625621  0.0726047     0.265096  0.265091  0.941555
 0.0657803  0.0696807  0.0681443  0.0697566     0.268201  0.264951  0.265451
 0.0708924  0.0726047  0.0695851  0.070082      0.259905  0.259179  0.256042
 0.0642673  0.0703597  0.0671434  0.0689784  …  0.265441  0.26524   0.249489
 0.0710863  0.0688534  0.0617956  0.0691195     0.252654  0.268085  0.272011
 0.0726047  0.0669959  0.0713116  0.06

In [16]:
function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    Moments = zeros(S, 9)
    
    μ = simulation_parameters[3]
    for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.cram_down = Results[s, :, 6];
            DF.Ds = Results[s,:,7];
            DF.Dj = Results[s,:,8];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = size(INCOURT_REORG,1)/ size(INCOURT,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)
        
        
        
            #9. percentage of cases where a cram down occurred inside court
            mm9 = mean(INCOURT.cram_down)
        
            
        
        
            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8, mm9]

            
            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

        end
    return vec(mean(Moments, dims=1))
end



  

simulate_moments (generic function with 1 method)

In [21]:
#testando se aumentar S muda a simulação


using Random

Random.seed!(2021)

#test
S = 100
N = 66

simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

#conclusão: aumentar o número de simulações acima de 40 tem efeitos apenas na terceira casa decimal de alguns momentos

9-element Array{Float64,1}:
 1.559042507036741
 0.9988800855668556
 2.73053054094131
 0.9509090909090909
 0.736512271563752
 0.49932144805262707
 0.7672495779670531
 0.5564624109950596
 0.1515483107872748

Calculando a derivada dos momentos em relação aos parâmetros

In [20]:
#data_moments
data_moments = [1.479, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497, 0.0938];

In [38]:
function simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(round(θs0, digits=0));
    θj0 = Int64(round(θj0, digits=0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

simulate_new_moments (generic function with 1 method)

In [43]:
# calculando a derivada em relação a ρ

function derivative_moments(S, N, game_parameters, simulation_parameters, simulated_moments, data1, data2, n1, aux)
    
    
    P = size(game_parameters,1)+size(simulation_parameters,1)
    M = size(simulated_moments,1)
    D = zeros(M, P)
    
    estimated_game_parameters = game_parameters
    estimated_simulation_parameters = simulation_parameters
    
    #delta for each parameter
    #approximately 1% or the lowest decimal point
    #[game_parameters, simulation_parameters]
    delta = [0.01, 0.09, 0.005, -0.01, 0.01,  1.0, 1.0, 0.06]
    
    
    for i in 1:P
        if(i<=size(game_parameters,1))
            game_parameters = copy(estimated_game_parameters)
            
            game_parameters[i] = game_parameters[i] + delta[i]
            
            
        else
            simulation_parameters = copy(estimated_simulation_parameters)
            simulation_parameters[i-size(game_parameters,1)] = simulation_parameters[i-size(game_parameters,1)] + delta[i]
            
            
        end
        
        
        A2 = @spawnat aux solve_tree(data2, game_parameters)
        A1 = solve_tree(data1, game_parameters)
        A2 = fetch(A2)
        
        
        new_simulated_moments = simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
        
        #normalizing by the simulated moments before the change in the parameter
        D[:, i] = (new_simulated_moments .- simulated_moments ) ./ simulated_moments
        
    end
    
    
    
    return D
    
end


derivative_moments (generic function with 1 method)

In [44]:
Derivative = derivative_moments(S, N, game_parameters, simulation_parameters, simulated_moments, data1, data2, n1, aux)

  8.652416 seconds (30.02 M allocations: 642.090 MiB, 1.85% gc time)
        From worker 3:	  9.708780 seconds (30.02 M allocations: 642.090 MiB)
7.640519 seconds (30.02 M allocations: 642.090 MiB)
 10.557823 seconds (30.02 M allocations: 642.090 MiB, 1.25% gc time)
        From worker 3:	 20.302523 seconds (30.02 M allocations: 642.090 MiB, 55.14% gc time)
9.648953 seconds (30.02 M allocations: 642.090 MiB, 1.62% gc time)
       From worker 3:	  9.540754 seconds (30.02 M allocations: 642.090 MiB, 1.26% gc time)
10.208541 seconds (30.02 M allocations: 642.090 MiB, 1.44% gc time)
       From worker 3:	 10.268279 seconds (30.02 M allocations: 642.090 MiB, 1.15% gc time)
10.306896 seconds (30.02 M allocations: 642.090 MiB, 1.39% gc time)
       From worker 3:	 10.416096 seconds (30.02 M allocations: 642.090 MiB, 1.11% gc time)
10.603341 seconds (30.02 M allocations: 642.090 MiB)
       From worker 3:	 10.961350 seconds (30.02 M allocations: 642.090 MiB, 1.16% gc time)
10.971823 seconds (3

       From worker 3:	 13.565024 seconds (30.02 M allocations: 642.090 MiB, 0.51% gc time)
13.110687 seconds (30.02 M allocations: 642.090 MiB, 0.95% gc time)
 12.974262 seconds (30.02 M allocations: 642.090 MiB, 0.95% gc time)
       From worker 3:	 13.458232 seconds (26.96 M allocations: 595.398 MiB, 1.05% gc time)
      From worker 3:	193.370973 seconds (477.26 M allocations: 9.987 GiB, 0.84% gc time)
10.667705 seconds (26.96 M allocations: 595.398 MiB, 1.15% gc time)
204.496924 seconds (537.31 M allocations: 11.241 GiB, 1.03% gc time)
parameter 3[0.91, 9.4, 0.055, 0.99, 0.37][22.0, 36.0, 6.31]
  6.470531 seconds (30.02 M allocations: 642.090 MiB, 0.98% gc time)
  7.303749 seconds (30.02 M allocations: 642.090 MiB, 4.89% gc time)
  7.418269 seconds (30.02 M allocations: 642.090 MiB, 2.22% gc time)
  7.663744 seconds (30.02 M allocations: 642.090 MiB)
        From worker 3:	 34.711018 seconds (30.02 M allocations: 642.090 MiB, 72.33% gc time)
9.875044 seconds (30.02 M allocations: 64

       From worker 3:	 12.758607 seconds (30.02 M allocations: 642.090 MiB, 0.75% gc time)
12.155408 seconds (30.02 M allocations: 642.090 MiB, 1.01% gc time)
       From worker 3:	 12.929556 seconds (30.02 M allocations: 642.090 MiB, 0.68% gc time)
12.497008 seconds (30.02 M allocations: 642.090 MiB, 1.07% gc time)
       From worker 3:	 12.857304 seconds (30.02 M allocations: 642.090 MiB, 0.65% gc time)
13.028803 seconds (30.02 M allocations: 642.090 MiB, 1.04% gc time)
       From worker 3:	 13.425097 seconds (26.96 M allocations: 595.398 MiB, 0.72% gc time)
      From worker 3:	186.550322 seconds (477.26 M allocations: 9.987 GiB, 0.90% gc time)
12.179170 seconds (26.96 M allocations: 595.398 MiB, 0.97% gc time)
197.096384 seconds (537.31 M allocations: 11.242 GiB, 1.03% gc time)
parameter 6[0.91, 9.4, 0.05, 0.99, 0.38][23.0, 36.0, 6.31]
  7.777745 seconds (30.02 M allocations: 642.090 MiB, 0.74% gc time)
       From worker 3:	  9.815374 seconds (30.02 M allocations: 642.090 MiB, 0.

9×8 Array{Float64,2}:
  0.0384818     0.00260228    0.0129724    …  -0.00706756   0.00654735
  0.000163961   7.68569e-6   -0.000270893      0.0         -0.000148666
  0.0750925     0.00533287    0.0266638       -0.0106667    0.00563368
  0.000801667  -0.00945968   -0.661055        -0.00625301  -0.00256534
 -0.0210938    -0.0343889    -0.181599        -0.0257748   -0.0790342
 -0.0359994    -0.0569159    -0.242262     …  -0.0369293   -0.0403106
  0.00580154   -0.00149105    0.0484291        0.00535448   0.00121216
  0.0201652     0.000436334   0.748287         0.0167035   -0.000575506
  0.187683     -0.0376611    -0.0148077       -0.0419559    0.0661108

In [56]:
Derivative[2,:]

8-element Array{Float64,1}:
  0.00016396130513200903
  7.685686178113238e-6
 -0.0002708925910876333
  2.6445371796040528e-6
  0.00016396130513200903
  0.00016396130513200903
  0.0
 -0.00014866625786446135

In [46]:
using DelimitedFiles

writedlm( "Derivative-cd50.csv",  Derivative, ',')